In [1]:
import numpy as np
import random
import time
import operator
from genetic_algorithm import ga

In [2]:
# Set general parameters
chromosome_length = 100
population_size = 10
#best_score_progress = [] # Tracks progress

run_dict = {}
reliable_num = 0

while(reliable_num < 25):
    population_size_array = np.array([])
    new_population_size = population_size
    crossover_operator = '2X'
    bisection_search = 0
    success_population_sizes = np.array([])
    #success_population = np.array([]).reshape(0,100)
    num_generation = 0
    num_fitnessfunc = 0
    while(True):
        start = time.time()
        population = ga.create_starting_population(int(new_population_size), chromosome_length)
        outcome = ga.run_generation(population, crossover_operator)
        num_fitnessfunc += 1
        population_size_array = np.hstack((population_size_array,new_population_size))

        if(outcome == "Fail" and new_population_size <= 2560):
            #print(f'Fail - (population size: {len(population)})')
            num_generation += 1
            if(bisection_search == 0):
                new_population_size = new_population_size*2
            else:
                midpoint_gap = abs((population_size_array[-1] - population_size_array[-2]))/2
                if(midpoint_gap < 10):
                    break
                new_population_size =  population_size_array[-1] + midpoint_gap    

        else:
            #print(f'Success - (population size: {len(population)})')
            num_generation += 1
            success_population_sizes = np.hstack((success_population_sizes,new_population_size))
            #success_population = np.vstack((success_population,outcome.reshape(1,100)))
            if(len(population_size_array)==1):
                break
            bisection_search = 1
            midpoint_gap = abs((population_size_array[-1] - population_size_array[-2]))/2
            if(midpoint_gap < 10):
                break
            new_population_size = population_size_array[-1] - midpoint_gap 
            population = ga.create_starting_population(int(new_population_size), chromosome_length)
    end = time.time()
    elapsed_time = end - start
    optimal_population_size = success_population_sizes[-1]
    #optimal_population = success_population[-1] 
    
    
    if (str(int(optimal_population_size)) not in run_dict.keys()):
        run_dict[str(int(optimal_population_size))] = [0,0,0,0,0]
    run_dict[str(int(optimal_population_size))][0] += 1
    run_dict[str(int(optimal_population_size))][1] += num_generation
    run_dict[str(int(optimal_population_size))][2] += num_fitnessfunc
    run_dict[str(int(optimal_population_size))][3] += elapsed_time
    
    run_dict_sorted = dict(sorted(run_dict.items(), key=operator.itemgetter(1),reverse=True))
    reliable_num = run_dict_sorted[(list(run_dict_sorted.keys())[0])][0]
    
    #print(f'\nOptimal population size: {optimal_population_size}')
    #print(f'Number of generations: {num_generation}')
    #print(f'Optimal population: {optimal_population}')
    
    
   
    #print(f'Time elapsed: {elapsed_time}')

C:\Users\Rohit\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


NameError: name 'end' is not defined

In [7]:
sorted_d[list(sorted_d.keys())[0]]

[25, 100, 100, 0.21912002563476562, 0]

In [ ]:
a = {}
a["10"] = [10, 3]
a["20"] = [22, 1]
a

In [ ]:
import operator
sorted_d = dict(sorted(a.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
sorted_d[(list(sorted_d.keys())[0])][0]

In [ ]:

sorted_d = dict(sorted(d.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
list(sorted_d.keys())[0]

In [ ]:
# Plot progress
import matplotlib.pyplot as plt
plt.plot(best_score_progress)
plt.xlabel('Generation')
plt.ylabel('Best score (% target)')
plt.show()